**Test for washout filtering**

# Load Data

In [1]:
import pandas as pd
import numpy as np

from Sequential_Fish.pipeline_parameters import RUN_PATH, WASHOUT_KEY_WORD
from Sequential_Fish.tools import safe_merge_no_duplicates

In [2]:
WASHOUT_KEY_WORD

'Washout'

In [3]:
Acquisition = pd.read_feather(RUN_PATH + '/result_tables/Acquisition.feather')
Detection = pd.read_feather(RUN_PATH + '/result_tables/Detection.feather')
Clusters = pd.read_feather(RUN_PATH + '/result_tables/Clusters.feather')
Spots = pd.read_feather(RUN_PATH + '/result_tables/Spots.feather')
Gene_map = pd.read_feather(RUN_PATH + '/result_tables/Gene_map.feather')

In [4]:
Gene_map['is_washout'] = Gene_map['target'].str.contains(WASHOUT_KEY_WORD)
Gene_map

,map_id,cycle,color_id,target,is_washout
0,0,0,0,POLR2A,False
1,1,1,0,Washout_1_0,True
2,2,2,0,POLR2B,False
3,3,3,0,Washout_3_0,True
4,4,4,0,POLR2C,False
5,5,5,0,Washout_5_0,True
6,6,6,0,POLR2E,False
7,7,7,0,Washout_7_0,True
8,8,8,0,POLR2F,False
9,9,9,0,Washout_9_0,True


In [5]:
Detection = safe_merge_no_duplicates(
    Detection,
    Acquisition,
    keys= ['cycle'],
    on= 'acquisition_id'
)
Spots = safe_merge_no_duplicates(
    Spots,
    Detection,
    keys=['cycle','color_id'],
    on= 'detection_id'
)
Spots = safe_merge_no_duplicates(
    Spots,
    Gene_map,
    keys='is_washout',
    on= ['cycle','color_id']
)

In [6]:
Spots['coordinates'] = list(zip(Spots['z'], Spots['y'], Spots['x']))
Spots

,spot_id,cluster_id,z,y,x,intensity,population,detection_id,cycle,color_id,is_washout,coordinates
0,0,NaN,0,0,597,5927,free,1,0,0,False,"(0, 0, 597)"
1,1,NaN,0,77,389,6095,free,1,0,0,False,"(0, 77, 389)"
2,2,NaN,0,141,993,6923,free,1,0,0,False,"(0, 141, 993)"
3,3,NaN,0,144,1107,6960,free,1,0,0,False,"(0, 144, 1107)"
4,4,NaN,0,265,673,7111,free,1,0,0,False,"(0, 265, 673)"
...,...,...,...,...,...,...,...,...,...,...,...,...
339004,287933,NaN,6,812,174,8032,free,174,5,0,True,"(6, 812, 174)"
339005,287934,0.0,0,1218,755,17217,clustered,174,5,0,True,"(0, 1218, 755)"
339006,287935,0.0,0,1217,757,16242,clustered,174,5,0,True,"(0, 1217, 757)"
339007,287936,0.0,0,1217,755,16993,clustered,174,5,0,True,"(0, 1217, 755)"


In [7]:
cycle_list = list(Spots['cycle'].unique())
cycle_list.sort()

banned_coordinates = []
for cycle in cycle_list :
    print("cycle ",cycle)
    new_washout_idx = Spots.loc[
        (Spots['coordinates'].isin(banned_coordinates)) & (Spots['cycle'] == cycle)
        ].index
    print(f"{len(new_washout_idx)} new spots added to washout")

    Spots.loc[new_washout_idx, ['is_washout']] = True

    new_banned_coordinates = list(
        Spots[
            (Spots['is_washout']) & (Spots['cycle'] == cycle)
            ]['coordinates'].unique()
    )
    print(f"{len(new_banned_coordinates)} new banned positions")

    banned_coordinates += new_banned_coordinates
    banned_coordinates = list(pd.unique(banned_coordinates))
    print(f"{len(banned_coordinates)} currently banned positions\n")


cycle  0
0 new spots added to washout
0 new banned positions
0 currently banned positions

cycle  1
0 new spots added to washout
72 new banned positions
72 currently banned positions

cycle  2
3 new spots added to washout
3 new banned positions
72 currently banned positions

cycle  3
4 new spots added to washout
17 new banned positions
85 currently banned positions

cycle  4
5 new spots added to washout
5 new banned positions
85 currently banned positions

cycle  5
1 new spots added to washout
36 new banned positions
120 currently banned positions

cycle  6
7 new spots added to washout
6 new banned positions
120 currently banned positions

cycle  7
2 new spots added to washout
136 new banned positions
255 currently banned positions

cycle  8
12 new spots added to washout
9 new banned positions
255 currently banned positions

cycle  9
6 new spots added to washout
441 new banned positions
695 currently banned positions

cycle  10
14 new spots added to washout
12 new banned positions
695 